## Accessing the X8 Machine Tutorial


In this notebook you will find the steps to connect to and submit jobs to Xanadu's hardware from their cloud computing services.

#### Generate API Key

First, go to https://cloud.xanadu.ai and create an account. Then, you'll need to generate an API Key to connect to their machines. This can be achieved by navigating to your API Keys page (as shown below).

![image](Gen_API_Key.png) 

#### Connect to the Machine

To connect to, and use the Xanadu X8 photonic device, you must first install the strawberryfields and Xanadu Cloud Client (XCC) python packages. You may do so by running the following commands in your notebook or terminal

In [1]:
# Omit the "!" if you are downloading the packages from your terminal
!pip install strawberryfields
!pip install xcc

Then, to connect remotely to the X8 machine run the following block of code:

In [3]:
import xcc
xcc.Settings(REFRESH_TOKEN="INSERT TOKEN HERE").save()

To check your connection to the machine, you can run the following command:

In [4]:
import xcc.commands
xcc.commands.ping()

'Successfully connected to the Xanadu Cloud.'

#### Congratulations! You're connected to the X8 Chip!

Now that you're connected to the photonic device, you can run experiments remotely using Xanadu's [strawberry fields](https://strawberryfields.ai) package. However, there are limitations. Below you can find the architecture of the X8 chip. Any circuits you'd like to evaluate on the chip must adhere to the restrictions below:

- The initial states are two-mode squezed states, where qumodes 0-3 are the signal modes, and qumodes 4-7 are the idler modes. The pairs are (0,4),(1,5),(2,6),(3,7). Note: the S2 gate (that generate the two-mode states between these pairs) only corrently supports (r=1, phi=0) and (r=0, phi=0) (no squeezing)
- Arbitrary 4x4 unitary operation must be applied to both the signal modes and idler modes. The unitary consists of the operations BSgate, MZgate, Rgate and Interferometer. This is where we get creative freedom to implement what we want.
- Non-gaussian photon counting measurement is performed at the end of the evolution.


<img src="X8Arch.png" alt="drawing" width="600"/>



#### How to run a quantum circuit on X8

Adhereing to the restrictions of the X8 chip, we can create a quantum circuit using strawberry fields and then submit the job remotely. First we import the necessary packages

In [5]:
import numpy as np

import strawberryfields as sf
from strawberryfields import ops
from strawberryfields import RemoteEngine

Then, we create an 8-qumode program (the object that strawberry fields uses to encode quantum circuits):

In [6]:
prog = sf.Program(8, name="my_job")

Then we can create a quantum circuit as below:

In [7]:
with prog.context as q:
    # First, we generate the two-mode squeezed states using the S2 Gates.
    ops.S2gate(1.0) | (q[0], q[4])
    ops.S2gate(1.0) | (q[1], q[5])
    ops.S2gate(1.0) | (q[3], q[7])

    # Then, we apply a couple gates on our signal qumodes
    ops.BSgate(0.543, 0.123) | (q[2], q[0])
    ops.Rgate(0.453) | q[1]
    ops.MZgate(0.65, -0.54) | (q[2], q[3])

    # The same circuit must be applied to the idler modes.
    ops.BSgate(0.543, 0.123) | (q[6], q[4])
    ops.Rgate(0.453) | q[5]
    ops.MZgate(0.65, -0.54) | (q[6], q[7])
    
    # measure using photon-counting 
    ops.MeasureFock() | q

Now all that is left to do is submit the job. We choose our engine:

In [8]:
eng = RemoteEngine("X8")

Then we run our program on our selected backend:

In [9]:
results = eng.run(prog, shots=1000)

2023-09-14 13:48:00,630 - INFO - Compiling program for device X8_01 using compiler Xunitary.
2023-09-14 13:48:19,533 - INFO - The remote job fc9bf0a3-0720-4f70-984f-765e67d0887b has been completed.


Then we can analyze the results of our job. The results will tell us how much photons were detected in each mode:

In [10]:
print(results.samples)

[[0 0 1 ... 1 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 1 ... 0 1 0]
 [1 2 2 ... 0 0 0]
 [1 1 0 ... 0 0 0]]


We can also determine the mean amount of photons found in each mode:

In [12]:
print(np.mean(results.samples, axis=0))

[0.329 0.451 0.351 0.122 0.32  0.435 0.371 0.185]
